## Imports

In [ ]:
import pdfplumber
import pandas as pd
import numpy as np
from datetime import datetime
import os

## Functions

In [ ]:
def get_date(str_date):
    global year
    date = datetime.strptime(str_date+year, '%d-%b%Y')
    return date

def is_date(date):
    try:
        get_date(date)
        return True
    except ValueError:
        return False

def is_float(str_float):
    try:
        float(str_float)
        if str_float[-3] == ".":
            return True
        else:
            return False
    except ValueError:
        return False

In [ ]:
def enter_row_into_statement_dict(date, details, amount, balance):
    global statement_dict
    
    statement_dict["Date"] += [date]
    statement_dict["Transaction Details"] += [details]
    statement_dict["Amount"] += [amount]
    statement_dict["Balance"] += [balance]
    
def clear_statement_dict():
    global statement_dict
    statement_dict["Date"] = []
    statement_dict["Transaction Details"] = []
    statement_dict["Amount"] = []
    statement_dict["Balance"] = []
    
def clear_statement_globals():
    global balance
    global amount
    global details
    global date
    
    date = ""
    balance = ""
    amount = ""
    details = ""

In [ ]:
def check_if_two_transactions(items):
    global balance
    global amount
    global details
    
    second_item_is_number = is_float(items[-2])
    
    if second_item_is_number:
        balance = items[-1]
        amount = items[-2]
        details = " ".join(items[1:-2])
    else:
        amount = items[-1]
        details = " ".join(items[1:-1])
    
def check_if_new_transaction(items):
    global date
    global balance
    global details
    global amount
    
    last_item_is_number = is_float(items[-1])
    details_exist = details != ""
    starts_with_date = is_date(items[0])
    
    if last_item_is_number:
        if details_exist:
            if starts_with_date:
                enter_row_into_statement_dict(date,details,amount,balance)
                date = get_date(items[0])  
                check_if_two_transactions(items)
            
            else:
                enter_row_into_statement_dict(date, details, amount, "")
                check_if_two_transactions(items)

        else:
            date = get_date(items[0])
            details = " ".join(items[1:-1])
            balance = items[-1].replace(",","")
    else:
        details = details + " " + " ".join(items)
    
def prepare_statement_dict(statement):
    for lines in statement:
        row_items = lines.split()
        check_if_new_transaction(row_items)
        
    enter_row_into_statement_dict(date,details,"",balance)

In [ ]:
def check_statement_for_dates(statement):
    global year
    
    first_date = statement[0].split()[0]
    last_date = statement[-1].split()[0]

    if not(is_date(first_date) & is_date(last_date)):
        print("Statement Dates not found")
        print(first_date)

def get_statement(invoice, text):
    global year
    
    year = invoice[invoice.find('/')+1:invoice.find('-')]
    start = [i for i, s in enumerate(text.split("\n")) if 'B/F BALANCE' in s]
    end = [i for i, s in enumerate(text.split("\n")) if 'C/F BALANCE' in s]

    statement = [row.replace(",", "") for row in text.split("\n")[start[0]:end[0]+1]]
    check_statement_for_dates(statement)
    return statement

In [ ]:
def convert(invoices):
    global statement_dict
    
    for invoice in invoices:
        with pdfplumber.open(invoice) as pdf:
            page = pdf.pages[0]
            text = page.extract_text(x_tolerance=1)

        statement = get_statement(invoice, text)
        prepare_statement_dict(statement)
        clear_statement_globals()
        
    df = pd.DataFrame(statement_dict, columns= ["Date", "Transaction Details", "Amount", "Balance"])
    return df

## Global Variables

In [ ]:
year = ""

In [ ]:
statement_dict = {
    "Date": [],
    "Transaction Details": [], 
    "Amount": [],
    "Balance": [] 
}

In [ ]:
date = ""
balance = ""
amount = ""
details = ""

# ==========================================

## START HERE

In [ ]:
invoices = [f'statements/{string}'for string in os.listdir(f'{os.getcwd()}/statements') if string[0] != '.']
invoices

In [ ]:
df = convert(invoices).sort_values(by="Date")

In [ ]:
df.to_csv("statement.csv", index=False)

# ==========================================

## Clean Up

In [ ]:
clear_statement_dict()
# statement_dict